In [1]:
import cv2
import numpy as np
import glob
from tqdm import tqdm

#设置阈值
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
#预设棋盘格角点数
w = 6
h = 9
#角点世界坐标，以mm为单位
worldp= np.zeros((w*h,3), np.float32)
worldp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)*20

# 储存棋盘格角点的世界坐标和图像坐标对
worldpoints = [] # 在世界坐标系中的三维点
imgpoints = [] # 在图像平面的二维点

gray = None
images = glob.glob('pic6/*.jpg')
#用于文件命名
#ii = 0
for fname in tqdm(images):

    img = cv2.imread(fname)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 调用findChessboardCorners查找角点
    ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
    # 如果找到角点组，保存起来
    if ret == True:
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        worldpoints.append(worldp)
        imgpoints.append(corners)
        
        # 将角点绘制在棋盘格上
        #cv2.drawChessboardCorners(img, (w,h), corners, ret)

        #cv2.imwrite('test/calibresult_'+str(ii)+'.jpg',img)
        #cv2.waitKey(1)
        ##ii+=1
        
cv2.destroyAllWindows()
#标定相机，获取内参矩阵和畸变系数
ret, mtx, dist, _, _ = cv2.calibrateCamera(worldpoints, imgpoints, gray.shape[::-1], None, None)
print(("ret:"),ret)
print(("mtx:\n"),mtx)        # 内参数矩阵
print(("dist:\n"),dist)      # 畸变系数   distortion cofficients = (k_1,k_2,p_1,p_2,k_3)

# 反畸变
img_test = cv2.imread("test_pic.jpg")
h,w = img_test.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),0,(w,h)) # 自由比例参数
img_dst = cv2.undistort(img_test, mtx, dist, None, newcameramtx)
# 根据兴趣区域裁切图片
x,y,w,h = roi
img_dst = img_dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.jpg',img_dst)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:06<00:00,  4.87it/s]


ret: 0.3842940149003152
mtx:
 [[3.12757530e+03 0.00000000e+00 1.02648203e+03]
 [0.00000000e+00 3.12744985e+03 5.06658577e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
 [[-5.32539241e-02  1.97503055e+01 -1.36881263e-04  3.43796125e-03
  -5.23363192e+02]]


True